In [2]:
# Import libraries------------------------------------------------------------------------------------------------------

import os
import numpy as np 
import pandas as pd 
import napari
import re

from scipy import ndimage as nd
from tifffile import imread, imwrite
from skimage.measure import label, regionprops 
from skimage.segmentation import expand_labels
from skimage.morphology import binary_erosion, square, opening
from tqdm import tqdm

In [2]:
base_path = 'Z:/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/fgf4_cdx2_embryo/05_17'

In [4]:
tmp_list = os.listdir(base_path)
img_list = []

for tmp in tmp_list:
    if re.search('_561nm.tif', tmp):
        img_list.append(tmp)

print(img_list)
print(len(img_list))

['240519_iDHB_FGF_day1_fixed_cdx2_fgf_001_561nm.tif', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_002_561nm.tif', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_003_561nm.tif', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_004_561nm.tif', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_005_561nm.tif', '240519_iDHB_FGF_day1_fixed_cdx2_nofgf_001_561nm.tif', '240519_iDHB_FGF_day1_fixed_cdx2_nofgf_002_561nm.tif', '240519_iDHB_FGF_day1_fixed_cdx2_nofgf_003_561nm.tif']
8


In [10]:
img_name = img_list[0]
img_path = os.path.join(base_path, img_name)

img = imread(img_path)
print(img.shape)

(32, 2200, 2200)


### Editing masks

In [40]:
img_name = img_list[7]
img_path = os.path.join(base_path, img_name)

mask_name = img_name.replace('.tif', '_cp_masks.tif')
mask_path = os.path.join(base_path, mask_name)

img = imread(img_path)
mask = imread(mask_path)

In [41]:
viewer = napari.view_image(img, blending = 'additive')
mask_layer = viewer.add_labels(mask, opacity = 0.25)

In [38]:
mask = viewer.layers['mask'].data
mask = np.uint16(mask)
imwrite(mask_path, mask)

embryo = viewer.layers['Labels'].data
embryo = np.uint8(embryo)
embryo_path = os.path.join(base_path, img_name.replace('.tif', '_embryo.tif'))
imwrite(embryo_path, embryo)

In [39]:
viewer.close()

### Create cytoplasm and analyze

In [3]:
# Define functions--------------------------------------------------------------

def cyto_expand(img, px): 
    cyto_shell = expand_labels(img, distance = px) 
    cyto_buffer = expand_labels(img, distance = 2) 
    cyto = np.subtract(cyto_shell, cyto_buffer) 

    return cyto 

def roi_quant(mask_img, embryo_img, ch405_img, ch488_img, ch561_img):
    df = pd.DataFrame()

    id = []
    embryo_id = []
    area = []
    ch405_int = []
    ch488_int = []
    ch561_int = []

    embryo_props = regionprops(mask_img, intensity_image = embryo_img)
    ch405_props = regionprops(mask_img, intensity_image = ch405_img)
    ch488_props = regionprops(mask_img, intensity_image = ch488_img)
    ch561_props = regionprops(mask_img, intensity_image = ch561_img)

    for roi in range(len(embryo_props)):
        id.append(embryo_props[roi].label)
        embryo_id.append(round(embryo_props[roi].mean_intensity, 0))
        area.append(embryo_props[roi].area)
        ch405_int.append(ch405_props[roi].mean_intensity)
        ch488_int.append(ch488_props[roi].mean_intensity)
        ch561_int.append(ch561_props[roi].mean_intensity)

    df['id'] = id
    df['embryo_id'] = embryo_id
    df['area'] = area
    df['ch405_int'] = ch405_int
    df['ch488_int'] = ch488_int
    df['ch561_int'] = ch561_int

    return df

In [4]:
base_path = '/Volumes/LECIMAGE/Analysis/[NCI] [LGI] Sergio Ruiz/Bechara Saykali/images/fgf4_cdx2_embryo/05_17'

In [5]:
tmp_list = os.listdir(base_path)
img_list = []

for tmp in tmp_list:
    if re.search('_405nm.tif', tmp):
        img_list.append(tmp.replace('_405nm.tif', ''))

print(img_list)
print(len(img_list))

['240519_iDHB_FGF_day1_fixed_cdx2_fgf_005', '240519_iDHB_FGF_day1_fixed_cdx2_nofgf_002', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_003', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_001', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_004', '240519_iDHB_FGF_day1_fixed_cdx2_nofgf_001', '240519_iDHB_FGF_day1_fixed_cdx2_fgf_002', '240519_iDHB_FGF_day1_fixed_cdx2_nofgf_003']
8


In [11]:
for img_name in img_list:
    print(img_name)
    
    mask_path = os.path.join(base_path, img_name + '_561nm_cp_masks.tif')
    mask_img = imread(mask_path)

    embryo_path = os.path.join(base_path, img_name + '_561nm_embryo.tif')
    embryo_img = imread(embryo_path)

    ch405_path = os.path.join(base_path, img_name + '_405nm.tif')
    ch405_img = imread(ch405_path)

    ch488_path = os.path.join(base_path, img_name + '_488nm.tif')
    ch488_img = imread(ch488_path)

    ch561_path = os.path.join(base_path, img_name + '_561nm.tif')
    ch561_img = imread(ch561_path)

    cyto_img = np.empty([mask_img.shape[0], mask_img.shape[1], mask_img.shape[2]])
    df_nuc = pd.DataFrame()
    df_cyto = pd.DataFrame()
    
    for z_plane in tqdm(range(mask_img.shape[0])):
        mask_frame = mask_img[z_plane, :, :]
        embryo_frame = embryo_img[z_plane, :, :]
        ch405_frame = ch405_img[z_plane, :, :]
        ch488_frame = ch488_img[z_plane, :, :]
        ch561_frame = ch561_img[z_plane, :, :]

        cyto_frame = cyto_expand(mask_frame, 12)
        cyto_frame = np.uint16(cyto_frame)
        
        df_tmp = roi_quant(mask_frame, embryo_frame, ch405_frame, ch488_frame, ch561_frame)
        df_tmp['z_plane'] = z_plane
        df_nuc = pd.concat([df_nuc, df_tmp])

        df_tmp = roi_quant(cyto_frame, embryo_frame, ch405_frame, ch488_frame, ch561_frame)
        df_tmp['z_plane'] = z_plane
        df_cyto = pd.concat([df_cyto, df_tmp])

        cyto_frame = np.expand_dims(cyto_frame, axis = 0)
        cyto_img = np.concatenate((cyto_img, cyto_frame), axis = 0)

    cyto_img = np.uint16(cyto_img)
    cyto_path = os.path.join(base_path, img_name + '_cyto.tif')
    imwrite(cyto_path, cyto_img)

    df_nuc_path = os.path.join(base_path, img_name + '_output.csv')
    df_nuc.to_csv(df_nuc_path, header = True)

    df_cyto_path = os.path.join(base_path, img_name + '_cyto.csv')
    df_cyto.to_csv(df_cyto_path, header = True)

240519_iDHB_FGF_day1_fixed_cdx2_fgf_005


100%|███████████████████████████████████████████| 22/22 [00:13<00:00,  1.68it/s]


240519_iDHB_FGF_day1_fixed_cdx2_nofgf_002


100%|███████████████████████████████████████████| 24/24 [00:15<00:00,  1.53it/s]


240519_iDHB_FGF_day1_fixed_cdx2_fgf_003


...

100%|███████████████████████████████████████████| 21/21 [00:11<00:00,  1.82it/s]


240519_iDHB_FGF_day1_fixed_cdx2_fgf_002


100%|███████████████████████████████████████████| 32/32 [00:26<00:00,  1.22it/s]


240519_iDHB_FGF_day1_fixed_cdx2_nofgf_003


100%|███████████████████████████████████████████| 25/25 [00:14<00:00,  1.68it/s]


### Visualization

In [6]:
img_name = img_list[0]
print(img_name)

mask_path = os.path.join(base_path, img_name + '_561nm_cp_masks.tif')
mask_img = imread(mask_path)

cyto_path = os.path.join(base_path, img_name + '_cyto.tif')
cyto_img = imread(cyto_path)

240519_iDHB_FGF_day1_fixed_cdx2_fgf_005


In [7]:
viewer = napari.view_image(mask_img, blending = 'additive')
cyto_layer = viewer.add_labels(cyto_img)

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


TypeError: Only integer types are supported for Labels layers, but data contains float64.